In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark
import pandas as pd

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# Ejercicio 37

In [24]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('redirect_list.csv', header=True, inferSchema=True)
rdd = df.rdd

In [25]:
redirectByTitle = rdd.map(lambda x: (x.rd_title, 1))

In [26]:
redirectByTitle.reduceByKey(lambda x,y: x+y).reduce(lambda x,y: x if x[1] > y[1] else y)

('Artículo_futuro', 1161)

El contenido al que más se hacen redirecciones es 'Artículo_futuro'.

# Ejercicio 13 

In [27]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('geo_tags.csv', header=True, inferSchema=True)
rdd = df.rdd

In [28]:
withRegions = rdd.filter(lambda x: x.gt_country != None and x.gt_region != None)

In [29]:
byCountryAndRegion = withRegions.map(lambda x: ((x.gt_country, x.gt_region), 1))
byCountryAndRegion = byCountryAndRegion.reduceByKey(lambda x,y: x+y)

In [30]:
byCountry = byCountryAndRegion.map(lambda x: (x[0][0],(x[0][1],x[1])))

In [32]:
topByCountry = byCountry.reduceByKey(lambda x,y: x if x[1] > y[1] else y)

In [33]:
topByCountry.collect()

[('CH', ('BS', 10)),
 ('CN', ('11', 10)),
 ('RU', ('MOW', 5)),
 ('TZ', ('12', 1)),
 ('ER', ('MA', 2)),
 ('NO', ('5', 2)),
 ('EE', ('39', 2)),
 ('PR', ('DIF', 1)),
 ('KZ', ('BAY', 1)),
 ('FR', ('75', 8)),
 ('IN', ('DL', 4)),
 ('GH', ('AA', 1)),
 ('IL', ('JM', 4)),
 ('KE', ('300', 1)),
 ('SK', ('BL', 1)),
 ('IE', ('L', 8)),
 ('NP', ('CN', 1)),
 ('NZ', ('NTL', 1)),
 ('CS', ('SR', 1)),
 ('LK', ('71', 1)),
 ('SD', ('26', 2)),
 ('GT', ('PE', 2)),
 ('IS', ('1', 1)),
 ('IR', ('7', 2)),
 ('SB', ('WE', 1)),
 ('PH', ('CEB', 2)),
 ('PL', ('MA', 3)),
 ('GR', ('A1', 2)),
 ('AU', ('WA', 9)),
 ('BG', ('23', 2)),
 ('DO', ('11', 1)),
 ('TL', ('LA', 1)),
 ('KP', ('8', 3)),
 ('EG', ('C', 1)),
 ('AN', ('ES', 1)),
 ('JM', ('1', 2)),
 ('TR', ('34', 6)),
 ('DE', ('BE', 51)),
 ('PT', ('10', 1)),
 ('BS', ('FC', 1)),
 ('PE', ('PAS', 7)),
 ('MR', ('1', 1)),
 ('BA', ('BIH', 4)),
 ('CY', ('3', 1)),
 ('UA', ('43', 1)),
 ('CR', ('H', 1)),
 ('EH', ('GI', 2)),
 ('ET', ('OR', 1)),
 ('SI', ('61', 1)),
 ('NL', ('UT', 1)),

En la lista de arriba se ven las regiones por pais con mas contenidos publicados

# Ejercicio 26

In [34]:
df = sqlContext.read.csv('contents.csv', header=True, inferSchema=True)
rdd = df.rdd.cache()

In [35]:
sinRevision = rdd.filter(lambda x: x.parent_revision == None and x.id != None ).count()

In [36]:
totales = rdd.filter(lambda x: x.id != None ).count()
(sinRevision / totales) * 100

35.89491724601234

El porcentaje de que no tengan parent_revision (anteultima edicion) es de 35.89491724601234 ignorando los que no tienen id del contenido.

# Ejercicio 12

In [18]:
conf = pyspark.SparkConf().setAll([("spark.executor.heartbeatInterval","3600s"),( "spark.network.timeout","4000s")])
sc.stop()
sc = pyspark.SparkContext(conf=conf)

In [37]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('geo_tags.csv', header=True, inferSchema=True)
rdd = df.rdd

In [39]:
edicionesMarcianas = rdd.filter(lambda x: x.gt_globe == 'mars')

In [40]:
edicionesMarcianas.count()

15

In [41]:
localizacionEditMars = edicionesMarcianas.map(lambda x: (int(x.gt_id),(x.gt_lat, x.gt_lon)))

In [42]:
import math
distanciasEuclideas = localizacionEditMars.cartesian(localizacionEditMars).map(lambda x: ((x[0][0],x[1][0]),math.sqrt((x[0][1][1]-x[1][1][1])**2+(x[0][1][0]-x[1][1][0])**2))).cache()

distanciasEuclideas es la 'matriz' que contiene las distancias entre los distintos contenidos que estan en Marte.

In [43]:
distanciasEuclideas.filter(lambda x: x[0][0] != x[0][1]).reduce(lambda x,y: x if x[1] < y[1] else y)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 23.0 failed 1 times, most recent failure: Lost task 5.0 in stage 23.0 (TID 140) (host.docker.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\igna-\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 599, in main
  File "C:\Users\igna-\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 562, in read_int
    length = stream.read(4)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.2800.0_x64__qbz5n2kfra8p0\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:545)
	at org.apache.spar

Esto encontraria la menor distancia, si anduviera. Aclaracion: el error lo consulte y se me dijo que lo deje asi.

# Ejercico 28

In [15]:
conf = pyspark.SparkConf().setAll([("spark.executor.heartbeatInterval","3600s"),( "spark.network.timeout","4000s")])
sc.stop()
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('contents.csv', header=True, inferSchema=True)
rdd = df.rdd

In [16]:
sinNone = rdd.filter(lambda x: x.revisor_id != None and x.revisor_comment == None )

In [17]:
sinComentById = sinNone.map(lambda x: (x.revisor_id,1)).reduceByKey(lambda x,y: x+y).cache()
total = sinComentById.count()

In [18]:
cantidadCon1Edit = sinComentById.filter(lambda x: x[1] == 1).count()
cantidadCon10Edit = sinComentById.filter(lambda x: x[1] > 10).count()
cantidadCon100Edit = sinComentById.filter(lambda x: x[1] > 100).count()

In [19]:
porcentaje1 = cantidadCon1Edit / total
porcentaje10 = cantidadCon10Edit / total
porcentaje100 = cantidadCon100Edit / total

In [20]:
porcentaje1 * 100

56.585306172386375

In [21]:
porcentaje10 * 100

13.24402030953692

In [22]:
porcentaje100 * 100

2.973022572949165

Cantidad Ediciones | Porcentaje
-------- | -------
1 | %56.58
> 10 | %13.24
> 100 | %2.97


# Ejercicio 21

In [10]:
!pip install pyarrow
logs = pd.read_csv('logs.csv')
logs.to_parquet('logs.parquet')
del logs
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet('logs.parquet')
rdd = df.rdd

C:\Users\igna-\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
byTypeAndTitle = rdd.filter(lambda x: x.logtype != None and x.title != None  ).map(lambda x: ((x.logtype, x.title), 1))

In [12]:
contados = byTypeAndTitle.reduceByKey(lambda x,y: x+y)

In [13]:
top = contados.map(lambda x: (x[0][0], (x[0][1],x[1])))
top.take(5)

[('delete', ('Rapunzel', 1)),
 ('upload', ('Archivo:La florida uoct.jpg', 1)),
 ('upload', ('Archivo:Pablo zalaquett.jpg', 1)),
 ('delete', ('Beat box', 1)),
 ('delete', ('Bituminoso', 1))]

In [14]:
top.reduceByKey(lambda x,y: x if x[1] > y[1] else y).collect()

[('renameuser', ('Especial:Renameuser', 102)),
 ('patrol', ('Yoshitoshi', 4)),
 ('instructor',
  ('Education Program:Wikimedia Venezuela/Servicio Comunitario UCAB (2017-1)',
   6)),
 ('protect', ('Plantilla:Ficha de entidad subnacional', 96)),
 ('institution', ('Education Program:Universidad Pablo de Olavide', 4)),
 ('create', ('Categoría:Nacidos en 2019', 22)),
 ('gblrights', ('Usuario:Dferg', 3)),
 ('managetags', ('Especial:Tags', 5)),
 ('delete', ('Especial:Registro/delete', 437)),
 ('campus',
  ('Education Program:Wikimedia Chile/Análisis de Información Documental I (2016-1)',
   4)),
 ('student',
  ('Education Program:Tec de Monterrey, Campus Ciudad de Mexico/Experiencias Retadoras CCM (ene-mayo 2015)',
   184)),
 ('online',
  ('Education Program:Wikimedia Venezuela/Servicio Comunitario UCAB (2017-1)',
   9)),
 ('course',
  ('Education Program:Tec de Monterrey, Campus Ciudad de Mexico/Wikiservicio (mayo-julio)',
   70)),
 ('block', ('Usuario:80.58.205.33', 38)),
 ('gblblock', ('Us

El vector de arriba es la respuesta de forma "logtype": ("contenido", "#acciones")